# Sentiment Analysis
In this notebook we will build a complete sentiment analysis model using the IMDB movie review dataset to classify them as either positive or negative.

We will follow these steps:
1.  **Import Libraries:** Load all necessary packages.
2.  **Load Data:** Read the `IMDB Dataset10M.csv` files.
3.  **Preprocess Text:** Clean the text (remove HTML, punctuation, stopwords) and tokenize it.
4.  **Build Vocabulary:** Create a word-to-index mapping from our training data.
5.  **Create PyTorch Datasets & DataLoaders:** Convert our data into a format PyTorch can use, including padding and numericalization.
6.  **Define the LSTM Model:** Create the neural network architecture, defining which embedding method is going to be use: Learnt Embedding or Glove trained embeddings (file "glove.6B.100d.txt" needed).
7.  **Train the Model:** Feed the data to the model and update its weights.
8.  **Evaluate the Model:** Check how well our model performs on unseen test data.
9.  **Run Inference:** Use the trained model to predict the sentiment of new, custom reviews.

## Step 1: Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Download NLTK data (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Added to download the missing resource

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## Step 2: Load Data

We'll load our training and testing data from the CSV files using pandas. The `imdb_tr.csv` file will be used for both training and validation, and `test/imdb_te.csv` will be reserved for our final test.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Load the IMDB Dataset.csv file
try:
    # Use engine='python' for better handling of parsing errors and on_bad_lines='warn' to log issues
    imdb_df = pd.read_csv('/content/IMDB Dataset10M.csv', engine='python', on_bad_lines='warn')
    # The original notebook uses a column named 's' for sentiment. If 'IMDB Dataset.csv' has 'sentiment', rename it.
    if 'sentiment' in imdb_df.columns and 's' not in imdb_df.columns:
        imdb_df = imdb_df.rename(columns={'sentiment': 's'})
    # Convert sentiment to numerical labels (0 for negative, 1 for positive)
    imdb_df['s'] = imdb_df['s'].map({'negative': 0, 'positive': 1})

except FileNotFoundError:
    print("Error: 'IMDB Dataset.csv' not found. Creating a dummy DataFrame for demonstration.")
    # Create a dummy DataFrame with sample data to prevent further errors
    data = {
        'review': [
            "This movie was absolutely fantastic, a must-watch for everyone!",
            "I truly hated this film, it was a complete waste of my time.",
            "It was an okay experience, nothing extraordinary, but not bad either.",
            "A brilliant performance by the lead actor, very inspiring and moving.",
            "The plot was incredibly predictable and the acting was just terrible.",
            "Highly recommend this, I was on the edge of my seat the whole time.",
            "So boring, I nearly fell asleep within the first fifteen minutes.",
            "Loved every single minute of it, definitely going to watch it again.",
            "Quite a disappointing experience, I expected much more from this.",
            "Mediocre at best, could have been significantly better in many aspects.",
            "What a masterpiece! The cinematography was stunning.",
            "This is the worst movie I've seen all year. Avoid at all costs.",
            "A decent effort, but it lacked a certain spark to make it great.",
            "Absolutely captivated me from start to finish, truly remarkable.",
            "Such a dull and unengaging story, felt like an eternity watching it."
        ],
        'sentiment': [
            "positive", "negative", "positive", "positive", "negative",
            "positive", "negative", "positive", "negative", "negative",
            "positive", "negative", "positive", "positive", "negative"
        ]
    }
    imdb_df = pd.DataFrame(data)
    # Ensure dummy data matches expected processing: rename sentiment to s and convert to numerical
    imdb_df = imdb_df.rename(columns={'sentiment': 's'})
    imdb_df['s'] = imdb_df['s'].map({'negative': 0, 'positive': 1})

# Assuming the text column is named 'review' in 'IMDB Dataset.csv', rename to 'text' for consistency
if 'review' in imdb_df.columns and 'text' not in imdb_df.columns:
    imdb_df = imdb_df.rename(columns={'review': 'text'})

print("Original Data Head:")
print(imdb_df.head())
print(f"Total samples in IMDB Dataset: {len(imdb_df)}")

# 2. Split imdb_df into train_val and test. Default values (80%/20%).
train_val_df, test_df = train_test_split(imdb_df,
                                        test_size=0.6, #def 0.2
                                        random_state=42,
                                        stratify=imdb_df['s'])


# 4. Print the first 5 rows of train_df, val_df, and test_df
print("\nTrain Data Head:")
print(train_val_df.head())
print("\nTest Data Head:")
print(test_df.head())

# 5. Print the total number of samples in train_df, val_df, and test_df
print(f"\nTotal training samples: {len(train_val_df)} ({(len(train_val_df)/len(imdb_df))*100:.2f}%) ")
print(f"Total test samples: {len(test_df)} ({(len(test_df)/len(imdb_df))*100:.2f}%) ")

Original Data Head:
                                                text  s
0  One of the other reviewers has mentioned that ...  1
1  A wonderful little production. <br /><br />The...  1
2  I thought this was a wonderful way to spend ti...  1
3  Basically there's a family where a little boy ...  0
4  Petter Mattei's "Love in the Time of Money" is...  1
Total samples in IMDB Dataset: 10000

Train Data Head:
                                                   text  s
9291  A bit "the movie in the movie" case, or as the...  1
6051  This utterly dull, senseless, pointless, spiri...  0
3778  Directed by E. Elias Merhige "Begotten" is an ...  0
4739  For getting so many positive reviews, this mov...  0
2973  When I first heard about the show, I heard a l...  1

Test Data Head:
                                                   text  s
9399  This is one of the best of the series, ranking...  1
8503  Such energy and vitality. You just can't go wr...  1
621   The jokes are obvious, the gags are

## Step 3: Preprocess Text

This is a crucial step. We need to clean the text to make it easier for the model to learn.
Our `preprocess_text` function will:
1.  Remove HTML tags (like `<br />`).
2.  Remove punctuation and special characters, keeping only letters.
3.  Convert all text to lowercase.
4.  Tokenize the text (split it into individual words).
5.  Remove common English stopwords (like 'a', 'the', 'is').

In [3]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # 1. Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # 2. Remove punctuation/special chars
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    # 3. Convert to lowercase
    text = text.lower()
    # 4. Tokenize
    tokens = word_tokenize(text)
    # 5. Remove stopwords
    processed_tokens = [word for word in tokens if word not in stop_words]
    return processed_tokens

# Let's test the function
sample_review = "This is a sample review... <br /><br />It's not bad, but it could be better! 10/10"
print(f"Original: {sample_review}")
print(f"Processed: {preprocess_text(sample_review)}")

Original: This is a sample review... <br /><br />It's not bad, but it could be better! 10/10
Processed: ['sample', 'review', 'bad', 'could', 'better']


## Step 4: Build Vocabulary

Our model can't understand words; it only understands numbers. We need to create a "vocabulary" that maps each unique word to an integer.

We'll add two special tokens:
* `<PAD>`: A token we'll use to pad shorter reviews so all sequences in a batch have the same length.
* `<UNK>`: A token for words that appear in our test data but not in our training data (unknown words).

In [4]:
def build_vocab(data, min_freq=5):
    word_counts = Counter()
    for text in data:
        word_counts.update(text)

    # Create vocabulary, starting with special tokens
    vocab = {'<PAD>': 0, '<UNK>': 1}

    # Add words that meet the minimum frequency
    idx = 2
    for word, count in word_counts.items():
        if count >= min_freq:
            vocab[word] = idx
            idx += 1
    return vocab

# Apply preprocessing to our training data before building vocab
print("Preprocessing training data (this may take a minute)...")
# We use train_val_df['text'] to build the vocab
processed_train_texts = [preprocess_text(text) for text in tqdm(train_val_df['text'])]

# Build the vocabulary
word2idx = build_vocab(processed_train_texts)
vocab_size = len(word2idx)

print(f"\nVocabulary size: {vocab_size}")
print("First 10 vocab items:", list(word2idx.items())[:10])

Preprocessing training data (this may take a minute)...


100%|██████████| 4000/4000 [00:03<00:00, 1148.45it/s]



Vocabulary size: 10194
First 10 vocab items: [('<PAD>', 0), ('<UNK>', 1), ('bit', 2), ('movie', 3), ('case', 4), ('theme', 5), ('virtual', 6), ('game', 7), ('reality', 8), ('even', 9)]


## Step 5: Create PyTorch Datasets & DataLoaders

Now we'll create a custom `SentimentDataset` class. This class will handle:
1.  Taking a review text.
2.  Preprocessing it.
3.  Converting its words to indices using our `word2idx` vocabulary.
4.  **Padding** or **truncating** the sequence so all sequences have the same `max_length`.

We'll also split our `train_val_df` into a training set and a validation set.

In [5]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, word2idx, max_length):
        self.texts = texts
        self.labels = labels
        self.word2idx = word2idx
        self.max_length = max_length
        self.stop_words = set(stopwords.words('english'))

    def __len__(self):
        return len(self.texts)

    def preprocess_text(self, text):
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
        text = text.lower()
        tokens = word_tokenize(text)
        processed_tokens = [word for word in tokens if word not in self.stop_words]
        return processed_tokens

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        processed_tokens = self.preprocess_text(text)

        # Convert to indices
        indexed_tokens = [self.word2idx.get(word, self.word2idx['<UNK>']) for word in processed_tokens]

        # Pad or truncate
        if len(indexed_tokens) < self.max_length:
            # Pad with <PAD> token (index 0)
            padded_tokens = indexed_tokens + [self.word2idx['<PAD>']] * (self.max_length - len(indexed_tokens))
        else:
            # Truncate
            padded_tokens = indexed_tokens[:self.max_length]

        return torch.tensor(padded_tokens), torch.tensor(label, dtype=torch.float32)

# --- Hyperparameters ---
MAX_LENGTH = 100 # def 200
BATCH_SIZE = 64  # def 64

# Split training data into train and validation sets
train_df, val_df = train_test_split(train_val_df, test_size=0.5, random_state=42, stratify=train_val_df['s'])

# Create Datasets
train_dataset = SentimentDataset(train_df['text'].tolist(), train_df['s'].tolist(), word2idx, MAX_LENGTH)
val_dataset = SentimentDataset(val_df['text'].tolist(), val_df['s'].tolist(), word2idx, MAX_LENGTH)
test_dataset = SentimentDataset(test_df['text'].tolist(), test_df['s'].tolist(), word2idx, MAX_LENGTH)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Train batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

Train batches: 32
Validation batches: 32
Test batches: 94


## Step 6: Define the LSTM Model and choose your embedding (random learnt or glove)

Time to build our network! It will have:
1.  `nn.Embedding`: Turns our word indices into dense vectors (embeddings).
2.  `nn.LSTM`: The main recurrent layer that processes the sequence.
3.  `nn.Linear`: A standard fully-connected layer to give us a final score.
4.  `nn.Sigmoid`: (Applied in the forward pass) To squash the score between 0 and 1.

In [18]:
import torch
import torch.nn as nn
import numpy as np


# ---------------------------------------------------------
# 1. SAFE GloVe Loader (skips malformed lines)
# ---------------------------------------------------------
def load_glove_embeddings(glove_path, word2idx, embedding_dim):
    """
    Loads pretrained GloVe vectors into a matrix aligned with word2idx.
    - Skips malformed or corrupted lines
    - Prints how many lines were invalid
    """
    vocab_size = len(word2idx)

    # Initialize with random embeddings for missing words
    embedding_matrix = np.random.uniform(
        -0.05, 0.05, (vocab_size, embedding_dim)
    ).astype(np.float32)

    print("Loading GloVe embeddings safely...")
    found = 0
    skipped = 0

    with open(glove_path, "r", encoding="utf8") as f:
        for i, line in enumerate(f):
            parts = line.strip().split()

            # A correct line must contain 1 word + embedding_dim values
            if len(parts) != embedding_dim + 1:
                skipped += 1
                continue

            word = parts[0]
            try:
                vector = np.asarray(parts[1:], dtype=np.float32)
            except ValueError:
                skipped += 1
                continue

            if word in word2idx:
                idx = word2idx[word]
                embedding_matrix[idx] = vector
                found += 1

    print(f"✔ Loaded {found} GloVe vectors.")
    print(f"⚠ Skipped {skipped} malformed lines.")
    print(f"Embedding matrix shape: {embedding_matrix.shape}")

    return torch.tensor(embedding_matrix)



# ---------------------------------------------------------
# 2. LSTM Model (supports GloVe or random embeddings)
# ---------------------------------------------------------
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                 n_layers, dropout, padding_idx,
                 glove_embeddings=None,
                 freeze_embeddings=False):

        super().__init__()

        # ----------- EMBEDDING LAYER -----------
        self.embedding = nn.Embedding(
            vocab_size,
            embedding_dim,
            padding_idx=padding_idx
        )

        if USE_GLOVE == 1:
            if glove_embeddings is None:
                raise ValueError("USE_GLOVE is 1 but glove_embeddings is None.")

            print("Using pretrained GloVe embeddings...")
            self.embedding.weight.data.copy_(glove_embeddings)

            if freeze_embeddings:
                self.embedding.weight.requires_grad = False
                print("GloVe embeddings are FROZEN (not trainable).")
            else:
                print("GloVe embeddings will be FINE-TUNED.")

        else:
            print("Using randomly initialized embeddings (train from scratch).")

        # ----------- LSTM LAYER -----------
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            bidirectional=True,
            dropout=dropout,
            batch_first=True
        )

        # ----------- DROPOUT -----------
        self.dropout = nn.Dropout(dropout)

        # ----------- OUTPUT LAYER -----------
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.sigmoid = nn.Sigmoid()


    def forward(self, text_batch):

        embedded = self.embedding(text_batch)
        lstm_out, (hidden, cell) = self.lstm(embedded)

        # Last forward and backward hidden states
        forward_hidden = hidden[-2, :, :]
        backward_hidden = hidden[-1, :, :]

        hidden_cat = torch.cat((forward_hidden, backward_hidden), dim=1)

        dropped = self.dropout(hidden_cat)
        logits = self.fc(dropped)
        output = self.sigmoid(logits)

        return output.squeeze()



# ---------------------------------------------------------
# 3. MODEL HYPERPARAMETERS
# ---------------------------------------------------------
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = 1
N_LAYERS = 2
DROPOUT = 0.5
PADDING_IDX = word2idx["<PAD>"]

# ---------------------------------------------------------
# 4. CHOOSE EMBEDDING OPTION
# ---------------------------------------------------------
USE_GLOVE = 1     # 0 = random, 1 = GloVe
# FREEZE = False    # freeze GloVe weights?
FREEZE = True    # freeze GloVe weights?

if USE_GLOVE == 1:
    glove_path = "/content/glove.6B.100d.txt"
    glove_embeddings = load_glove_embeddings(glove_path, word2idx, EMBEDDING_DIM)
else:
    glove_embeddings = None



# ---------------------------------------------------------
# 5. INSTANTIATE MODEL
# ---------------------------------------------------------
model = SentimentLSTM(
    vocab_size=vocab_size,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    output_dim=OUTPUT_DIM,
    n_layers=N_LAYERS,
    dropout=DROPOUT,
    padding_idx=PADDING_IDX,
    glove_embeddings=glove_embeddings,
    freeze_embeddings=FREEZE
)

print(model)


Loading GloVe embeddings safely...
✔ Loaded 9956 GloVe vectors.
⚠ Skipped 1 malformed lines.
Embedding matrix shape: (10194, 100)
Using pretrained GloVe embeddings...
GloVe embeddings are FROZEN (not trainable).
SentimentLSTM(
  (embedding): Embedding(10194, 100, padding_idx=0)
  (lstm): LSTM(100, 128, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


## Step 7: Train the Model

In [19]:
import torch.optim as optim
import torch.nn as nn
import torch # Added torch for device setup
from tqdm import tqdm # Added tqdm import, assuming it's used in training loop

# --- Training Hyperparameters ---
LEARNING_RATE = 0.001 #def:0.001
N_EPOCHS = 5 #def:5

# Loss function and optimizer
criterion = nn.BCELoss() # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Training on device: {device}")

model = model.to(device)
criterion = criterion.to(device)

def get_accuracy(preds, y):
    """Returns accuracy per batch"""
    # Round predictions to the closest integer (0 or 1)
    rounded_preds = torch.round(preds)
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

print("Starting training...")

for epoch in range(N_EPOCHS):

    train_loss = 0.0
    train_acc = 0.0

    # --- Training Phase ---
    model.train() # Set model to training mode

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{N_EPOCHS} [Train]"):
        inputs, labels = inputs.to(device), labels.to(device)

        # 1. Zero gradients
        optimizer.zero_grad()

        # 2. Forward pass
        predictions = model(inputs)

        # 3. Calculate loss and accuracy
        loss = criterion(predictions, labels)
        acc = get_accuracy(predictions, labels)

        # 4. Backward pass
        loss.backward()

        # 5. Update weights
        optimizer.step()

        train_loss += loss.item()
        train_acc += acc.item()

    # --- Validation Phase ---
    val_loss = 0.0
    val_acc = 0.0

    model.eval() # Set model to evaluation mode
    with torch.no_grad(): # Disable gradient calculation
        for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{N_EPOCHS} [Val]"):
            inputs, labels = inputs.to(device), labels.to(device)

            predictions = model(inputs)

            loss = criterion(predictions, labels)
            acc = get_accuracy(predictions, labels)

            val_loss += loss.item()
            val_acc += acc.item()

    # Print epoch statistics
    avg_train_loss = train_loss / len(train_loader)
    avg_train_acc = train_acc / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    avg_val_acc = val_acc / len(val_loader)

    print(f'Epoch {epoch+1:02} | Train Loss: {avg_train_loss:.3f} | Train Acc: {avg_train_acc*100:.2f}% | Val. Loss: {avg_val_loss:.3f} | Val. Acc: {avg_val_acc*100:.2f}%')

print("Training finished.")

Training on device: cpu
Starting training...


Epoch 1/5 [Val]: 100%|██████████| 32/32 [00:11<00:00,  2.72it/s]


Epoch 01 | Train Loss: 0.680 | Train Acc: 56.84% | Val. Loss: 0.633 | Val. Acc: 64.55%


Epoch 2/5 [Val]: 100%|██████████| 32/32 [00:12<00:00,  2.50it/s]


Epoch 02 | Train Loss: 0.586 | Train Acc: 70.31% | Val. Loss: 0.589 | Val. Acc: 70.56%


...

Epoch 03 | Train Loss: 0.553 | Train Acc: 73.39% | Val. Loss: 0.531 | Val. Acc: 74.76%


Epoch 4/5 [Val]: 100%|██████████| 32/32 [00:11<00:00,  2.67it/s]


Epoch 04 | Train Loss: 0.516 | Train Acc: 75.24% | Val. Loss: 0.520 | Val. Acc: 75.34%


Epoch 5/5 [Val]: 100%|██████████| 32/32 [00:11<00:00,  2.82it/s]

Epoch 05 | Train Loss: 0.477 | Train Acc: 77.83% | Val. Loss: 0.511 | Val. Acc: 76.81%
Training finished.


## Step 8: Evaluate the Model

Now we'll use the held-out test set (`test_loader`) to see how our model generalizes to completely new data.

In [20]:
test_loss = 0.0
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)

        predictions = model(inputs)

        loss = criterion(predictions, labels)
        test_loss += loss.item()

        # Store predictions and labels
        rounded_preds = torch.round(predictions)
        all_preds.extend(rounded_preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

avg_test_loss = test_loss / len(test_loader)
test_acc = accuracy_score(all_labels, all_preds)

print(f'Test Loss: {avg_test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
print("\nClassification Report:")
# 0 is negative, 1 is positive
print(classification_report(all_labels, all_preds, target_names=['Negative', 'Positive']))

Testing: 100%|██████████| 94/94 [00:33<00:00,  2.81it/s]

Test Loss: 0.504 | Test Acc: 76.63%

Classification Report:
              precision    recall  f1-score   support

    Negative       0.75      0.79      0.77      2983
    Positive       0.78      0.74      0.76      3017

    accuracy                           0.77      6000
   macro avg       0.77      0.77      0.77      6000
weighted avg       0.77      0.77      0.77      6000



## Step 9: Run Inference

Let's create a final function that takes any review as a string and predicts its sentiment.

In [9]:
def predict_sentiment(review_text, model, word2idx, max_length, device):
    model.eval()

    # Preprocess the text
    processed_tokens = preprocess_text(review_text)

    # Convert to indices
    indexed_tokens = [word2idx.get(word, word2idx['<UNK>']) for word in processed_tokens]

    # Pad or truncate
    if len(indexed_tokens) < max_length:
        padded_tokens = indexed_tokens + [word2idx['<PAD>']] * (max_length - len(indexed_tokens))
    else:
        padded_tokens = indexed_tokens[:max_length]

    # Convert to tensor and add batch dimension (batch size = 1)
    input_tensor = torch.tensor(padded_tokens).unsqueeze(0).to(device)

    with torch.no_grad():
        prediction = model(input_tensor)

    probability = prediction.item()
    sentiment = "Positive" if probability > 0.5 else "Negative"

    return sentiment, probability

# --- Test with custom reviews ---

positive_review = "This movie was fantastic! The acting was superb and the plot was gripping. I would recommend this to everyone."
neg_review = "What a waste of time. The plot was predictable and the acting was terrible. I would not watch this again."

sentiment, prob = predict_sentiment(positive_review, model, word2idx, MAX_LENGTH, device)
print(f"Review: '{positive_review}'")
print(f"Sentiment: {sentiment} (Probability: {prob:.4f})\n")

sentiment, prob = predict_sentiment(neg_review, model, word2idx, MAX_LENGTH, device)
print(f"Review: '{neg_review}'")
print(f"Sentiment: {sentiment} (Probability: {prob:.4f})")

Review: 'This movie was fantastic! The acting was superb and the plot was gripping. I would recommend this to everyone.'
Sentiment: Positive (Probability: 0.6117)

Review: 'What a waste of time. The plot was predictable and the acting was terrible. I would not watch this again.'
Sentiment: Positive (Probability: 0.6107)
